# 05 - About Multiplication in RLWE
---
There are two methods of multiplication

* Utilize $\textsf{Relinearization}$
* Utilize $\textsf{RGSW}$ 

And we will try to component-wise multiplication via $\textsf{(I)FFT}$

---

In [1]:
import numpy as np
from numpy.polynomial import Polynomial

---

Parameters

In [2]:
n     = 16
q     = 7681
std   = 3.2
delta = 256
xi    = np.exp(2 * np.pi * 1j / 2*n)

---

First of all, we will try to component-wise multiplication via Fourier Transformation and rescaling technique such as, BG(F)V and CKKS.

We set dimension $n=16$ which means that it use $2n$-th cyclotomic ring.

* $\xi = \exp(2 \pi i / 2n) = \exp(\pi i / n)$

In [3]:
def componentwise_multiply_via_cyclic_convolution(a, b):
    N = len(a)
    # 입력 a와 b가 시간 영역에 있다고 보면, 먼저 ifft를 취해서 “쌍대 영역”으로 보냄
    X = np.fft.ifft(a)
    Y = np.fft.ifft(b)
    
    div = np.zeros(N+1)
    div[0] = -1
    div[-1] = 1

    P = np.polymul(X[::-1], Y[::-1])
    q, r = np.polydiv(P, div)

    # X와 Y의 순환 합성곱을 계산
    # Z = np.zeros(N, dtype=complex)
    # for k in range(N):
    #     for j in range(N):
    #         Z[k] += X[j] * Y[(k - j) % N]

    # 합성곱 결과에 fft를 취하면, 컨볼루션 정리(Convolution Theorem)에 의해
    # fft(Z) = fft(X) ⊙ fft(Y) = a ⊙ b (성분별 곱)이 됨.
    # result = np.fft.fft(Z)

    result = np.fft.fft(r[::-1])
    return result.real  # 만약 계수가 실수라면

# 예시 사용:
poly1_coef = np.array([1, 2, 3, 4])
poly2_coef = np.array([5, 6, 7, 8])
result = componentwise_multiply_via_cyclic_convolution(poly1_coef, poly2_coef)
print(result)  # 기대: [5, 12, 21, 32]


[ 5. 12. 21. 32.]


In [4]:
def componentwise_multiply_via_negacyclic_convolution(a, b):
    N = len(a)
    # 입력 a, b가 시간 영역의 계수라고 가정할 때,
    # 먼저 ifft를 취해 “쌍대 영역” (역변환 영역)로 옮김.
    X = np.fft.ifft(a)  # X = (1/N)*F⁻¹(a)
    Y = np.fft.ifft(b)  # Y = (1/N)*F⁻¹(b)
    
    # zeta = exp(pi*i/N): 2N차 원시 단위근 (zeta^N = -1)
    j = np.arange(N)
    zeta = np.exp(np.pi * 1j / N)
    
    # **쌍대 영역에서의 twisting:**  
    # 원래 fft 쪽에서는 곱셈 인자가 zeta^j였으므로,
    # ifft 쪽에서는 그 쌍대 효과로 zeta^(-j)를 곱해줍니다.
    X_twisted = X * (zeta ** (-j))
    Y_twisted = Y * (zeta ** (-j))
    
    div = np.zeros(N+1)
    div[0]  = 1
    div[-1] = 1

    P = np.polymul(X_twisted[::-1], Y_twisted[::-1])
    q, r = np.polydiv(P, div)
    r = r[::-1]

    # 부정순환 합성곱 (negacyclic convolution)을 계산
    # conv_neg = np.zeros(N, dtype=complex)
    # for m in range(N):
    #     s = 0
    #     for j_idx in range(N):
    #         # m - j_idx가 음수가 될 경우에는 "랩"하면서 부호가 바뀌어야 함
    #         if j_idx <= m:
    #             s += X_twisted[j_idx] * Y_twisted[m - j_idx]
    #         else:
    #             s += - X_twisted[j_idx] * Y_twisted[N + m - j_idx]
    #     conv_neg[m] = s

    # **Untwisting 및 스케일 보정:**  
    # 원래 untwisting에서는 fft 쪽에서 zeta^(-m)를 곱하고 1/N을 했으므로,
    # ifft 쪽에서는 그 쌍대 효과로 zeta^(m)를 곱하고 N을 곱해줍니다.
    # D = np.array([conv_neg[m] * (zeta ** (m)) for m in range(N)])
    D = np.array([r[m] * (zeta ** (m)) for m in range(N)])

    # 마지막에 fft를 취해 원래의 시간 영역(계수 영역)로 복원
    result = np.fft.fft(D)
    return result.real  # 계수가 실수라면

# 예시 사용:
poly1_coef = np.array([1, 2, 3, 4])
poly2_coef = np.array([5, 6, 7, 8])
result = componentwise_multiply_via_negacyclic_convolution(poly1_coef, poly2_coef)
print(result)  # 기대 결과: [5, 12, 21, 32]


[ 5. 12. 21. 32.]


In [5]:
a = np.array([1, 2, 3, 4])
b = np.array([5, 6, 7, 8])

a = np.concatenate((a, a[::-1]))
b = np.concatenate((b, b[::-1]))

a = np.fft.ifft(a)
b = np.fft.ifft(b)

# _a_conj = np.conjugate(_a)
# _b_conj = np.conjugate(_b)

# a = np.concatenate((_a, _a_conj[::-1]))
# b = np.concatenate((_b, _b_conj[::-1]))

a = np.concatenate((a.real, a.imag))
b = np.concatenate((b.real, b.imag))

# print(a)
# print(b)

div    = np.zeros(9)
div[0] =  -1
div[-1] =  1

q, r = np.polydiv(np.polymul(a[::-1],b[::-1]), div)

r_real = r[:3:-1]
r_imag = r[3::-1]
r_complex = r_real + 1j * r_imag

# print(r_complex)
print(np.fft.fft(r_complex).real)

# hmm .....


[13.68198052 25.61396103 20.81801948  8.38603897]


In [6]:
def vandermonde(xi: np.complex128, M: int) -> np.array:
    N = M // 2
    matrix = []
    for i in range(N):
        root = xi ** (2 * i + 1)
        row = []
        for j in range(N):
            row.append(root ** j)
        matrix.append(row)
    return matrix

def sigma_inverse(A, b: np.array) -> Polynomial:
    coeffs = np.linalg.solve(A, b)
    p = Polynomial(coeffs)
    return p

In [8]:
M = 8
N = 4
xi = np.exp(2 * np.pi * 1j / M) # 2N-th primitive roots of unity

mat = vandermonde(xi, M)
print(mat)

a = [9, 2]
a = np.concatenate((a, a[::-1]))

poly_a = sigma_inverse(mat, a)
coef = np.round(np.real(poly_a.coef)).astype(int)

print(coef)

print(np.fft.ifft(a))


[[(1+0j), (0.7071067811865476+0.7071067811865475j), (2.220446049250313e-16+1j), (-0.7071067811865474+0.7071067811865477j)], [(1+0j), (-0.7071067811865474+0.7071067811865477j), (-4.440892098500626e-16-1j), (0.707106781186548+0.707106781186547j)], [(1+0j), (-0.7071067811865479-0.7071067811865471j), (1.1102230246251565e-15+1j), (0.7071067811865464-0.7071067811865487j)], [(1+0j), (0.707106781186547-0.707106781186548j), (-1.3877787807814457e-15-1j), (-0.707106781186549-0.707106781186546j)]]
[ 5  2  0 -2]
[5.5 +0.j   1.75-1.75j 0.  +0.j   1.75+1.75j]


In [9]:
import numpy as np

def componentwise_multiply_separate_real_imag(a, b):
    N = len(a)
    
    # IFFT 변환
    X = np.fft.ifft(a)
    Y = np.fft.ifft(b)
    
    # 실수 및 허수 부분 분리
    X_r, X_i = X.real, X.imag
    Y_r, Y_i = Y.real, Y.imag

    # 복소수 곱셈의 실수 및 허수 부분 계산
    P_r = np.polymul(X_r[::-1], Y_r[::-1]) - np.polymul(X_i[::-1], Y_i[::-1])
    P_i = np.polymul(X_r[::-1], Y_i[::-1]) + np.polymul(X_i[::-1], Y_r[::-1])

    # 나눗셈을 위한 다항식 정의 (X^N - 1)
    div = np.zeros(N+1)
    div[0] = -1
    div[-1] = 1

    # 실수 및 허수 부분 나누기
    _, r_r = np.polydiv(P_r, div)
    _, r_i = np.polydiv(P_i, div)

    # FFT 변환 후 최종 결과 조합
    result = np.fft.fft(r_r[::-1]) + 1j * np.fft.fft(r_i[::-1])
    
    return result

# 테스트
poly1_coef = np.array([1, 2, 3, 4])
poly2_coef = np.array([5, 6, 7, 8])
result = componentwise_multiply_separate_real_imag(poly1_coef, poly2_coef)

print("실수 부분:", result.real)  # 기대: [5, 12, 21, 32]
print("허수 부분:", result.imag)  # 기대: [0, 0, 0, 0] (이론적으로)


실수 부분: [ 5. 12. 21. 32.]
허수 부분: [0. 0. 0. 0.]


In [87]:
import numpy as np

def componentwise_multiply_single_polynomial(a, b):
    N = len(a)

    # IFFT 변환 (계수를 복소수로 변환)
    X = np.fft.ifft(a)
    Y = np.fft.ifft(b)
    
    # 하나의 다항식에 인코딩: 짝수 차수에 실수, 홀수 차수에 허수
    Z = np.zeros(2 * N)
    W = np.zeros(2 * N)
    
    Z[::2]  = X.real    # 짝수 차수: 실수 부분
    Z[1::2] = X.imag   # 홀수 차수: 허수 부분
    W[::2]  = Y.real 
    W[1::2] = Y.imag 

    # 다항식 곱셈 수행
    P = np.polymul(Z[::-1], W[::-1])

    # X^(2N) - 1로 나누기
    div = np.zeros(2 * N + 1)
    div[0] =  -1
    div[-1] = 1
    _, r = np.polydiv(P, div)

    # 결과에서 실수와 허수 부분 복원
    R_real = np.zeros(N)
    R_imag = np.zeros(N)
    
    R_real = r[::-1][::2]  # 짝수 차수 계수 → 실수 부분
    R_imag = r[::-1][1::2]  # 홀수 차수 계수 → 허수 부분

    # FFT를 사용하여 최종 결과 변환
    result = np.fft.fft(R_real + 1j * R_imag)
    
    return result

# 테스트
poly1_coef = np.array([1, 2, 3, 4])
poly2_coef = np.array([5, 6, 7, 8])
result = componentwise_multiply_single_polynomial(poly1_coef, poly2_coef)

print("실수 부분:", result.real )  # 기대: [5, 12, 21, 32]
print("허수 부분:", result.imag )  # 기대: [0, 0, 0, 0] (이론적으로)


실수 부분: [ 5. 11. 21. 31.]
허수 부분: [ 0.  1.  0. -1.]


In [92]:
import numpy as np

def componentwise_multiply_single_polynomial(a, b):
    N = len(a)

    # IFFT 변환 (계수를 복소수로 변환)
    X = np.fft.ifft(a)
    Y = np.fft.ifft(b)
    
    # 하나의 다항식에 인코딩: 짝수 차수에 실수, 홀수 차수에 허수
    Z = np.zeros(2 * N)
    W = np.zeros(2 * N)
    
    Z[::2]  = X.real    # 짝수 차수: 실수 부분
    Z[1::2] = X.imag   # 홀수 차수: 허수 부분
    W[::2]  = Y.real 
    W[1::2] = Y.imag 

    # 다항식 곱셈 수행
    P = np.polymul(Z[::-1], W[::-1])

    # X^(2N) + 1로 나누기
    div = np.zeros(2 * N + 1)
    div[0] = -1
    div[-1] = 1  # 기존 X^(2N)-1에서 +1로 변경
    _, r = np.polydiv(P, div)

    # 부정순환곱 특성에 맞게 홀수 차수 항의 부호 반전
    r[1::2] = -r[1::2]

    # 결과에서 실수와 허수 부분 복원
    R_real = np.zeros(N)
    R_imag = np.zeros(N)
    
    R_real = r[::-1][::2]  # 짝수 차수 계수 → 실수 부분
    R_imag = r[::-1][1::2]  # 홀수 차수 계수 → 허수 부분

    # FFT를 사용하여 최종 결과 변환
    result = np.fft.fft(R_real + 1j * R_imag)
    
    return result

# 테스트
poly1_coef = np.array([1, 2, 3, 4])
poly2_coef = np.array([5, 6, 7, 8])
result = componentwise_multiply_single_polynomial(poly1_coef, poly2_coef)

print("실수 부분:", np.round(result.real, 6))  # 기대: [5, 12, 21, 32]
print("허수 부분:", np.round(result.imag, 6))  # 기대: [0, 0, 0, 0]


실수 부분: [ -5. -31. -21. -11.]
허수 부분: [ 0. -1.  0.  1.]
